In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')
RS = sum(list(map(ord, 'Dale Boca')))

import pandas as pd
pd.option_context('display.max_rows', None, 'display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

import numpy as np
import os;

## Lectura y Preparación del Dataset

In [7]:
data = pd.read_csv('dataset_final.csv', index_col=0)

In [8]:
from sklearn.preprocessing import LabelEncoder

def data_preparation(df, window=3, neutral=True):
    
    # armar las windows para el armado de las features 
    cols1 = ['pos', 'neg', 'neu'] if neutral==True else ['pos', 'neg']
    for col in cols1:
        df[col+'w'] = df[col].rolling(window=window).mean()
    
    # shifteo un día el retorno. las noticias deben estimar el retorno del día posterior.
    df['exc_ret']  = df['exc_ret'].shift(periods=-1)
    df['exc_ret%'] = df['exc_ret%'].shift(periods=-1) # por si lo necesito 
    
    # elimino las filas que quedan con nan por efecto del rolling window
    df.dropna(axis=0, inplace=True)
    
    # encoding del label que indica si hay retorno positivo (1) o negativo (0)
    le = LabelEncoder()
    df['exc_ret'] = le.fit_transform(df['exc_ret'])
    
    # split en train y test por fecha
    cols2 = [x+'w' for x in cols1]
    
    cut_date = '2018-10-31'
    
    X  = df.loc[df.date <= cut_date, cols2]
    y  = df.loc[df.date <= cut_date, 'exc_ret']
    
    Xt = df.loc[df.date > cut_date, cols2]
    yt = df.loc[df.date > cut_date, 'exc_ret']
    
    return X, Xt, y, yt

## Entrenamiento de Classifiers

In [9]:
import scipy.stats as st
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.svm import SVC

In [10]:
def train_classifier(X, Xt, y, yt):
    
    #model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Logistic Regression',
    #              'Catboost', 'Support Vector']
    model_name = ['Random Forest', 'XGBoost', 'LightGBM', 'Catboost']
   
    model_init = [RandomForestClassifier(),
                  XGBClassifier(),
                  lgb.LGBMClassifier(),
                  #LogisticRegression(),
                  CatBoostClassifier(),
                  #SVC(probability=True)
                 ]
    
    # Random Forest 
    params1 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth"   : st.randint(2, 25),     # Maximum tree depth for base learners.
    }
    
    # XGBoost y Lightgbm
    params2 = {  
        "n_estimators": st.randint(10,100),    # Number of boosted trees to fit.
        "max_depth": st.randint(2, 25),         # Maximum tree depth for base learners.
        "learning_rate": st.uniform(0.01, 0.5), # Boosting learning rate (xgb’s “eta”)
        "colsample_bytree": st.beta(10, 1),     # Subsample ratio of columns when constructing each tree.
        "subsample": st.beta(10, 1),            # Subsample ratio of the training instance.
        "gamma": st.uniform(0, 10),             # Minimum loss reduction required to make a further partition on a leaf node of the tree.
        'reg_alpha': st.uniform(0.05,10),       # L1 regularization term on weights
        "min_child_weight": st.uniform(1,20),   # Minimum sum of instance weight(hessian) needed in a child.
    }
    
    # Logistic Regression
    params3 = {  
        "penalty"     : ['l1', 'l2'],
        "C"           : st.uniform(0.1, 10.),
    }
    
    # Catboost
    params4 = {}
    
    # SVC
    params5 = {
        "C"           : st.uniform(0.1, 10.),
        #"kernel"      : ['lbf', 'linear']
    }
    
    #model_params = [params1, params2, params2, params3, params4, params5]
    model_params = [params1, params2, params2, params4]
    
    train_scores    = []
    test_scores     = []
    best_estimators = []
    best_parameters = []
    
    #kf    = KFold(n_splits=5, shuffle=True, random_state=RS)
    tsp   = TimeSeriesSplit(n_splits=6)
    
    for name, mod, params in zip(model_name, model_init, model_params):
        
        model = mod

        rgrid = RandomizedSearchCV(estimator=model, param_distributions=params, cv=tsp,
                                   scoring='roc_auc', n_iter=50, verbose=1, n_jobs=-1)
        rgrid.fit(X, y)
        
        best_estimators.append(rgrid.best_estimator_)
        best_parameters.append(rgrid.best_params_)
        train_scores.append(rgrid.best_score_)
        
        ppred = rgrid.predict_proba(Xt)
        score = roc_auc_score(y_score=ppred[:, 1], y_true=yt)
        
        test_scores.append(score)
    
    results = pd.DataFrame(
        {'Model'      : model_name,
         'Train Score': train_scores,
         'Test Score' : test_scores,
         'Params'     : best_parameters,
         'Estimator'  : best_estimators
        })
    
    return results

In [11]:
import itertools
windows = np.arange(1, 6)
neutral = [1, 0]

resultado = pd.DataFrame()

for win, neu in itertools.product(windows, neutral):
    
    X, X_test, y, y_test = data_preparation(data, window=win, neutral=neu)
    
    clfs = train_classifier(X, X_test, y, y_test)
  
    res = pd.DataFrame(
    {'Model'      : clfs['Model'],
     'Window'     : list(str(win)*clfs.shape[0]),
     'Neutral'    : list(str(neu)*clfs.shape[0]),
     'Train Score': clfs['Train Score'],
     'Test Score' : clfs['Test Score'],
     'Params'     : clfs['Params'],
     #'Estimator'  : clfs['Estimator']
    })
    
    resultado = pd.concat([resultado, res], axis=0)

Fitting 6 folds for each of 50 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    9.4s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 6 folds for each of 50 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 6 folds for each of 50 candidates, totalling 300 fits


...

177:	learn: 0.6119009	total: 1.62s	remaining: 7.48s
178:	learn: 0.6117385	total: 1.64s	remaining: 7.52s
179:	learn: 0.6114230	total: 1.65s	remaining: 7.52s
180:	learn: 0.6109415	total: 1.66s	remaining: 7.5s
181:	learn: 0.6106857	total: 1.67s	remaining: 7.49s
182:	learn: 0.6101596	total: 1.67s	remaining: 7.47s
183:	learn: 0.6100339	total: 1.68s	remaining: 7.45s
184:	learn: 0.6096510	total: 1.69s	remaining: 7.44s
185:	learn: 0.6092045	total: 1.7s	remaining: 7.42s
186:	learn: 0.6085723	total: 1.7s	remaining: 7.41s
187:	learn: 0.6083067	total: 1.71s	remaining: 7.39s
188:	learn: 0.6078644	total: 1.72s	remaining: 7.37s
189:	learn: 0.6076172	total: 1.73s	remaining: 7.35s
190:	learn: 0.6074099	total: 1.74s	remaining: 7.35s
191:	learn: 0.6070982	total: 1.74s	remaining: 7.34s
192:	learn: 0.6068286	total: 1.76s	remaining: 7.34s
193:	learn: 0.6065393	total: 1.76s	remaining: 7.33s
194:	learn: 0.6061773	total: 1.77s	remaining: 7.31s
195:	learn: 0.6055765	total: 1.78s	remaining: 7.3s
196:	learn: 0.60

349:	learn: 0.5685581	total: 3.21s	remaining: 5.96s
350:	learn: 0.5682327	total: 3.23s	remaining: 5.97s
351:	learn: 0.5682132	total: 3.24s	remaining: 5.96s
352:	learn: 0.5679673	total: 3.25s	remaining: 5.95s
353:	learn: 0.5678004	total: 3.25s	remaining: 5.94s
354:	learn: 0.5676935	total: 3.26s	remaining: 5.92s
355:	learn: 0.5674336	total: 3.27s	remaining: 5.91s
356:	learn: 0.5672671	total: 3.28s	remaining: 5.9s
357:	learn: 0.5668073	total: 3.28s	remaining: 5.89s
358:	learn: 0.5663953	total: 3.29s	remaining: 5.88s
359:	learn: 0.5662870	total: 3.3s	remaining: 5.87s
360:	learn: 0.5662179	total: 3.31s	remaining: 5.85s
361:	learn: 0.5660458	total: 3.31s	remaining: 5.84s
362:	learn: 0.5657808	total: 3.32s	remaining: 5.83s
363:	learn: 0.5656891	total: 3.33s	remaining: 5.82s
364:	learn: 0.5655534	total: 3.34s	remaining: 5.8s
365:	learn: 0.5654910	total: 3.34s	remaining: 5.79s
366:	learn: 0.5653305	total: 3.35s	remaining: 5.78s
367:	learn: 0.5650504	total: 3.36s	remaining: 5.77s
368:	learn: 0.5

514:	learn: 0.5373306	total: 4.6s	remaining: 4.33s
515:	learn: 0.5372706	total: 4.62s	remaining: 4.33s
516:	learn: 0.5370064	total: 4.64s	remaining: 4.33s
517:	learn: 0.5365282	total: 4.64s	remaining: 4.32s
518:	learn: 0.5364894	total: 4.65s	remaining: 4.31s
519:	learn: 0.5364318	total: 4.66s	remaining: 4.3s
520:	learn: 0.5363583	total: 4.67s	remaining: 4.29s
521:	learn: 0.5363474	total: 4.68s	remaining: 4.29s
522:	learn: 0.5361677	total: 4.69s	remaining: 4.28s
523:	learn: 0.5361406	total: 4.7s	remaining: 4.26s
524:	learn: 0.5359790	total: 4.7s	remaining: 4.25s
525:	learn: 0.5357421	total: 4.71s	remaining: 4.24s
526:	learn: 0.5356218	total: 4.72s	remaining: 4.24s
527:	learn: 0.5354837	total: 4.73s	remaining: 4.23s
528:	learn: 0.5353731	total: 4.74s	remaining: 4.22s
529:	learn: 0.5351549	total: 4.75s	remaining: 4.21s
530:	learn: 0.5351405	total: 4.75s	remaining: 4.2s
531:	learn: 0.5350935	total: 4.76s	remaining: 4.19s
532:	learn: 0.5349129	total: 4.78s	remaining: 4.18s
533:	learn: 0.534

685:	learn: 0.5107037	total: 5.99s	remaining: 2.74s
686:	learn: 0.5104226	total: 6.01s	remaining: 2.74s
687:	learn: 0.5103264	total: 6.01s	remaining: 2.73s
688:	learn: 0.5102542	total: 6.02s	remaining: 2.72s
689:	learn: 0.5101655	total: 6.03s	remaining: 2.71s
690:	learn: 0.5100954	total: 6.04s	remaining: 2.7s
691:	learn: 0.5099846	total: 6.05s	remaining: 2.69s
692:	learn: 0.5097302	total: 6.06s	remaining: 2.68s
693:	learn: 0.5096586	total: 6.06s	remaining: 2.67s
694:	learn: 0.5094756	total: 6.07s	remaining: 2.67s
695:	learn: 0.5094151	total: 6.08s	remaining: 2.66s
696:	learn: 0.5092225	total: 6.09s	remaining: 2.65s
697:	learn: 0.5090749	total: 6.1s	remaining: 2.64s
698:	learn: 0.5090135	total: 6.1s	remaining: 2.63s
699:	learn: 0.5089189	total: 6.11s	remaining: 2.62s
700:	learn: 0.5087891	total: 6.12s	remaining: 2.61s
701:	learn: 0.5086917	total: 6.13s	remaining: 2.6s
702:	learn: 0.5086770	total: 6.13s	remaining: 2.59s
703:	learn: 0.5086329	total: 6.14s	remaining: 2.58s
704:	learn: 0.50

857:	learn: 0.4863723	total: 7.36s	remaining: 1.22s
858:	learn: 0.4860120	total: 7.38s	remaining: 1.21s
859:	learn: 0.4859719	total: 7.38s	remaining: 1.2s
860:	learn: 0.4857213	total: 7.39s	remaining: 1.19s
861:	learn: 0.4854528	total: 7.4s	remaining: 1.18s
862:	learn: 0.4854281	total: 7.41s	remaining: 1.18s
863:	learn: 0.4852751	total: 7.41s	remaining: 1.17s
864:	learn: 0.4851434	total: 7.42s	remaining: 1.16s
865:	learn: 0.4850495	total: 7.43s	remaining: 1.15s
866:	learn: 0.4849371	total: 7.44s	remaining: 1.14s
867:	learn: 0.4845723	total: 7.44s	remaining: 1.13s
868:	learn: 0.4844594	total: 7.45s	remaining: 1.12s
869:	learn: 0.4844082	total: 7.46s	remaining: 1.11s
870:	learn: 0.4841828	total: 7.47s	remaining: 1.1s
871:	learn: 0.4839729	total: 7.47s	remaining: 1.1s
872:	learn: 0.4839509	total: 7.48s	remaining: 1.09s
873:	learn: 0.4839236	total: 7.49s	remaining: 1.08s
874:	learn: 0.4838659	total: 7.5s	remaining: 1.07s
875:	learn: 0.4838409	total: 7.5s	remaining: 1.06s
876:	learn: 0.4835

In [12]:
res = resultado.sort_values(by='Train Score', ascending=False)
res

,Model,Window,Neutral,Train Score,Test Score,Params
0,Random Forest,4,1,0.590965,0.490909,"{'max_depth': 16, 'n_estimators': 24}"
1,XGBoost,4,1,0.572966,0.454545,"{'colsample_bytree': 0.9414818544577265, 'gamma': 0.11592704627355288, 'learning_rate': 0.18121698995149615, 'max_depth': 17, 'min_child_weight': 2.784470346215705, 'n_estimators': 16, 'reg_alpha': 3.651116315540037, 'subsample': 0.9568127064418155}"
0,Random Forest,1,0,0.560053,0.538462,"{'max_depth': 8, 'n_estimators': 52}"
2,LightGBM,4,0,0.554578,0.340000,"{'colsample_bytree': 0.9223961272482367, 'gamma': 3.411109401966006, 'learning_rate': 0.5071559302415667, 'max_depth': 8, 'min_child_weight': 3.449843114428041, 'n_estimators': 65, 'reg_alpha': 0.856183992769306, 'subsample': 0.9077053224156402}"
2,LightGBM,1,0,0.548328,0.581731,"{'colsample_bytree': 0.9398713134175182, 'gamma': 3.2541429312099037, 'learning_rate': 0.18441212671423074, 'max_depth': 4, 'min_child_weight': 11.13421313924792, 'n_estimators': 30, 'reg_alpha': 2.697700509493418, 'subsample': 0.8704465477785911}"
2,LightGBM,4,1,0.544791,0.454545,"{'colsample_bytree': 0.9743645683196406, 'gamma': 0.8449753379533254, 'learning_rate': 0.16402460543089215, 'max_depth': 8, 'min_child_weight': 2.922741673296162, 'n_estimators': 73, 'reg_alpha': 3.4891387714145194, 'subsample': 0.8738087643297839}"
0,Random Forest,2,0,0.540435,0.512821,"{'max_depth': 23, 'n_estimators': 16}"
0,Random Forest,4,0,0.540123,0.320000,"{'max_depth': 13, 'n_estimators': 28}"
3,Catboost,1,0,0.538451,0.586538,{}
3,Catboost,4,1,0.537970,0.472727,{}


In [13]:
res.to_csv('resultados.csv')